<a href="https://colab.research.google.com/github/ysaidcan/federated-biomedical-informatics/blob/master/492_create_one_sec_sorted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas
from sklearn.preprocessing import Normalizer
import numpy as np
import os
import math
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import normalize as sk_normalize
from pickle import dump
from pickle import load
import random
from sklearn.utils import shuffle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
FOLDER_PATH = '/content/drive/MyDrive/empatica'
LABEL_FILE_PATH = '/content/drive/MyDrive/empatica/PSS - 6 (Scores) 10.06.19.csv'
EMPATICA_DATA_FOLDER = '/content/drive/MyDrive/empatica_data_new/'

WINDOW_SEC = 600 # Sec
SAMPLING_RATE = 1 # Hz
WINDOW_LENGTH = int(WINDOW_SEC * SAMPLING_RATE)
SKIP_SEC = int(WINDOW_SEC)

LABEL_NAMES = {2: 'Perceived score (L-H)', 3: 'Perceived Score (L-M-H)'}

SENSORS_TO_USE = ['ACC', 'BVP', 'EDA', 'TEMP']
SENSOR_HERTZ = {'ACC': 32, 'BVP': 64, 'EDA': 4, 'TEMP': 4}

# Size of X => n x len(WINDOW_LENGTH) x len(SENSORS_TO_USE)
# Size of y => n

In [ ]:
# Verilen window ve kisi IDsine gore "PSS - 6 (Scores) 10.06.19.csv" dosyasndan stress label'ini buluyor
df_scores = pandas.read_csv(LABEL_FILE_PATH)
def two_label_map(label:str) -> int:
    if label == 'L':
        return 0
    elif label == 'H':
        return 1

    print('LABEL ERROR: ' + str(label))
    return -1

def three_label_map(label:str) -> int:
    if label == 'L':
        return 0
    elif label == 'M':
        return 0
    elif label == 'H':
        return 1

    print('LABEL ERROR: ' + str(label))
    return -1

def convert_label(label:str) -> int:
    if LABEL_COUNT == 2:
        return two_label_map(label)
    return three_label_map(label)

def get_labels(id:int, utc_start:int) -> int:

    if id == 102:
      return (1,1)
    if id == 101 or id == 103:
      df_scores1 = pandas.read_csv('/content/drive/MyDrive/empatica_new/Labels.csv')
      utc_end = utc_start + WINDOW_SEC
    
      temp_df = df_scores1[df_scores1['Participant ID'] == id]
      temp_df = temp_df[temp_df['UTC_Start'] <= utc_start]
      temp_df = temp_df[temp_df['UTC_End'] >= utc_end]

      if len(temp_df) > 0:
          two_label = two_label_map(temp_df.iloc[0][LABEL_NAMES[2]])
          three_label = three_label_map(temp_df.iloc[0][LABEL_NAMES[3]])
          return (two_label, three_label)


    utc_end = utc_start + WINDOW_SEC
    
    temp_df = df_scores[df_scores['Participant ID'] == id]
    temp_df = temp_df[temp_df['UTC_Start'] <= utc_start]
    temp_df = temp_df[temp_df['UTC_End'] >= utc_end]

    if len(temp_df) > 0:
        two_label = two_label_map(temp_df.iloc[0][LABEL_NAMES[2]])
        three_label = three_label_map(temp_df.iloc[0][LABEL_NAMES[3]])
        return (two_label, three_label)
    return None

In [ ]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

def list_to_single(values):
    return values[0]

def pythagoras(values):
    return math.sqrt(values[0]**2 + values[1]**2 + values[2]**2)

# Sensor'un SENSOR_HERTZ'ini average alarak SAMPLING_RATE'e indiriyor
def sample_sensor_data(sensor:str, data:list) -> list:
    # len(data) = SENSOR_HERTZ[sensor] * WINDOW_SEC
    sampling_ratio = int(SENSOR_HERTZ[sensor] / SAMPLING_RATE)
    
    res_list = []

    for i in range(int(len(data) / sampling_ratio)):
        batch = data[i * sampling_ratio : (i + 1) * sampling_ratio]
        res_list.append(sum(batch) / len(batch))

    return res_list

In [ ]:
# Klasorlerde loop don, label'i var ise X'e kaydet

def create() -> pandas.DataFrame:
    df = pandas.DataFrame(columns=['EmpaticaID', 'SessionID', 'SessionLabel', 'SessionLabel2'] + [k + '_mean' for k in SENSORS_TO_USE])
    
    id_list = sorted(os.listdir(FOLDER_PATH))
    id_list2 = sorted(os.listdir('/content/drive/MyDrive/empatica_new'))
    id_list = id_list + id_list2
    for id in id_list:
        print(id)
  
        id_path = FOLDER_PATH + '/' + id
        if not os.path.isdir(id_path):
            continue
        session_list = sorted(os.listdir(id_path))
        for sess in session_list:

            sess_path = id_path + '/' + sess
            if not os.path.isdir(sess_path):
                continue
            # print(sess_path)
            sess_has_all_sensors = len(intersection(os.listdir(sess_path), [x + '.csv' for x in SENSORS_TO_USE])) == len(SENSORS_TO_USE)
            if not sess_has_all_sensors:
                continue

            df_sensors = {}
            # Find window count
            window_count = 0
            for sensor in SENSORS_TO_USE:
                df_sensors[sensor] = pandas.read_csv(sess_path + '/' + sensor +'.csv')

                sensor_windows_length = WINDOW_SEC * SENSOR_HERTZ[sensor]
                sensor_skip_length = SKIP_SEC * SENSOR_HERTZ[sensor]
                sensor_max_window_count = math.ceil((len(df_sensors[sensor]) - sensor_windows_length) / sensor_skip_length)

                if window_count == 0:
                    window_count = sensor_max_window_count
                else:
                    window_count = min(window_count, sensor_max_window_count)
            # print(sess.split('_')[1],  str(window_count))
            window_utc_start = int(sess[:10])
            for i in range(window_count):

                labels = get_labels(id=int(id), utc_start=window_utc_start)
                window_utc_start += SKIP_SEC
                
                if labels == None:
                    continue         
                
                # Bu window icin sensor data'larini oku, X'e ekle
                window_dict = {'EmpaticaID': [id for _ in range(WINDOW_SEC)], 'SessionID': [sess.split('_')[1] for _ in range(WINDOW_SEC)], 'SessionLabel': [labels[0] for _ in range(WINDOW_SEC)], 'SessionLabel2': [labels[1] for _ in range(WINDOW_SEC)]}
                for sensor in df_sensors:
                    df_sensor = df_sensors[sensor]
                    
                    sensor_windows_length = WINDOW_SEC * SENSOR_HERTZ[sensor]
                    sensor_skip_length = SKIP_SEC * SENSOR_HERTZ[sensor]
                    sensor_window_data_list = df_sensor[i * sensor_skip_length : i * sensor_skip_length + sensor_windows_length].values.tolist()
                    
                    if sensor == 'ACC': # sum of squares of X, Y and Z square-rooted
                        sensor_window_data_list = list(map(pythagoras, sensor_window_data_list))
                    else:
                        sensor_window_data_list = list(map(list_to_single, sensor_window_data_list))

                    sensor_window_data_list = sample_sensor_data(sensor=sensor, data=sensor_window_data_list)
                    
                    window_dict[sensor + '_mean'] = sensor_window_data_list
                
                df = df.append(other=pandas.DataFrame.from_dict(window_dict))

    return df

In [ ]:
df = create()
df.head()

1
10
11
12
13
14
15
3
4
5
6
7
8
9
PSS - 6 (Scores) 10.06.19.csv
PSS - 6 (Scores) 10.06.19.gsheet
101
102
103
Labels.csv


,EmpaticaID,SessionID,SessionLabel,SessionLabel2,ACC_mean,BVP_mean,EDA_mean,TEMP_mean
0,1,A012B4,0,0,63.887988,-4.135781,0.525486,32.750
1,1,A012B4,0,0,63.928384,4.445938,0.485454,32.765
2,1,A012B4,0,0,63.883930,3.243750,0.524845,32.770
3,1,A012B4,0,0,63.760831,-2.938281,0.524525,32.785
4,1,A012B4,0,0,63.568750,-4.781094,0.520682,32.790


In [ ]:
df.to_csv(EMPATICA_DATA_FOLDER + 'one_sec_sorted.csv')